In [32]:
from small_llm_reasoning.evaluation.gsm8k import questions, answers, get_score
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, AutoConfig
import os
from datasets import load_from_disk
from small_llm_reasoning.generation.vllm_generation import llama_forward
import pandas as pd
import json

cache_dir = '/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/transformers_cache/'
os.environ['TRANSFORMERS_CACHE'] = cache_dir


### Loading data

In [33]:
data_path= "../datasets/gsm8k"
data = load_from_disk(f'{data_path}/test/')
data

Dataset({
    features: ['question', 'answer'],
    num_rows: 1319
})

### Loading tokenizer

In [34]:
# Loading tokenizer
hf_token = os.getenv("hf_token")

model_name= "meta-llama/Llama-3.2-3B"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token, cache_dir=cache_dir)
tokenizer.pad_token_id = tokenizer.eos_token_id



### Preparing data

In [4]:
questions

['There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?',
 'If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?',
 'Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?',
 'Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?',
 'Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?',
 'There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?',
 'Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?',
 'Olivia has $23. She bought five bag

In [5]:
answers

['There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The final answer is 6',
 'There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The final answer is 5',
 'Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The final answer is 39',
 'Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8. The final answer is 8',
 'Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. The final answer is 9',
 'There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. The final answer is 29',
 'Michael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls. The final answer is 33',
 'Olivia had 23 d

In [6]:
def get_prompt(ex, few_shot):
    prompt=''
    
    if few_shot:
        for i in range(8):
            prompt+= f'\nGiven the following problem, reason and give a final answer to the problem.\nProblem: {questions[i]}\nYour response should end with "The final answer is [answer]" where [answer] is the response to the problem.\n'
            prompt+= f'\n{answers[i]}\n'
    
    prompt+= f'\nGiven the following problem, reason and give a final answer to the problem.\nProblem: {ex}\nYour response should end with "The final answer is [answer]" where [answer] is the response to the problem.\n'

    
    return prompt

def tokenize_function(example,few_shot):
    prompt= get_prompt(example['question'], few_shot)
    # prompt= tokenizer.apply_chat_template(prompt,  tokenize= False, add_generation_prompt=True)
    return {'input_ids': {'prompt_token_ids':tokenizer(prompt, add_special_tokens=False)['input_ids']}}


In [7]:
print(get_prompt(data['question'][0],True))


Given the following problem, reason and give a final answer to the problem.
Problem: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
Your response should end with "The final answer is [answer]" where [answer] is the response to the problem.

There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The final answer is 6

Given the following problem, reason and give a final answer to the problem.
Problem: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
Your response should end with "The final answer is [answer]" where [answer] is the response to the problem.

There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The final answer is 5

Given the following problem, reason and give a final answer to the problem.
Problem: Leah had 32 choco

In [8]:
# tokenized_dataset = data.map(tokenize_function, batched=False)
tokenized_dataset = data.map(lambda x: tokenize_function(x,few_shot=True), batched=False)


Map: 100%|██████████| 1319/1319 [00:02<00:00, 642.69 examples/s]


In [9]:
len(tokenized_dataset['input_ids'])

1319

In [10]:
tokenized_dataset

Dataset({
    features: ['question', 'answer', 'input_ids'],
    num_rows: 1319
})

In [11]:
tokenized_dataset.save_to_disk(f"{data_path}/tokenized/LLaMA3B/pretrained/test/eight-shot/")

Saving the dataset (1/1 shards): 100%|██████████| 1319/1319 [00:00<00:00, 63503.37 examples/s]


### Running vLLM inference

In [35]:
data= load_from_disk(f"{data_path}/tokenized/LLaMA3B/pretrained/test/eight-shot/")

In [13]:

max_tokens= 50
temperature= 0
n_samples= 1 
all_outputs = llama_forward(
        prompts=data['input_ids'], 
        model_path=model_name, 
        max_tokens=max_tokens, 
        temperature=temperature, 
        n_samples=n_samples,
        n_gpus=1,
    )

WARNING 03-05 20:34:42 arg_utils.py:862] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 03-05 20:34:42 config.py:999] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 03-05 20:34:42 llm_engine.py:213] Initializing an LLM engine (v0.6.0) with config: model='meta-llama/Llama-3.2-3B', speculative_config=None, tokenizer='meta-llama/Llama-3.2-3B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:00<00:00,  1.58it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:00<00:00,  2.48it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:00<00:00,  2.28it/s]


INFO 03-05 20:34:44 model_runner.py:926] Loading model weights took 6.0160 GB


INFO 03-05 20:34:45 gpu_executor.py:122] # GPU blocks: 19082, # CPU blocks: 2340
INFO 03-05 20:34:47 model_runner.py:1217] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 03-05 20:34:47 model_runner.py:1221] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 03-05 20:34:59 model_runner.py:1335] Graph capturing finished in 11 secs.


Processed prompts: 100%|██████████| 1319/1319 [04:09<00:00,  5.29it/s, est. speed input: 5602.84 toks/s, output: 2641.61 toks/s]


In [14]:
generated_outputs=[]
for ex_outputs in all_outputs:
    generated_outputs.append({
        "input": tokenizer.decode(ex_outputs.prompt_token_ids, skip_special_tokens=False), 
        "output": [
            ith_output.text for ith_output in ex_outputs.outputs
        ]    
    })


In [15]:
df=pd.DataFrame(generated_outputs)

In [16]:
output_path= '../outputs/exp-0/eval_6/generated_results.json'
eval_data_path= f"{data_path}/tokenized/LLaMA3B/pretrained/test/eight-shot/"

output_dir = os.path.dirname(output_path)
if output_dir:  # Check if there is a directory part in the path
    os.makedirs(output_dir, exist_ok=True)  # Creates directory if it doesn't exist

with open(output_path, "w") as f:
    json.dump(generated_outputs, f, indent=4)

score = get_score(eval_data_path,output_path)
print(f"SCORE of {model_name} : ",score)
print(f"Output saved in: {output_path}")

SCORE of meta-llama/Llama-3.2-3B :  0.23426838514025777
Output saved in: ../outputs/exp-0/eval_6/generated_results.json


In [30]:
print(df['input'][5])


Given the following problem, reason and give a final answer to the problem.
Problem: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
Your response should end with "The final answer is [answer]" where [answer] is the response to the problem.

There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The final answer is 6

Given the following problem, reason and give a final answer to the problem.
Problem: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
Your response should end with "The final answer is [answer]" where [answer] is the response to the problem.

There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The final answer is 5

Given the following problem, reason and give a final answer to the problem.
Problem: Leah had 32 choco

In [31]:
print(df['output'][5][0])

Hint: You can use the formula 2x = 1.2y to solve this problem.

Kylar wants to buy 16 glasses. If he buys one glass for 5 dollars, then he will have to pay 5 x 16 = 80 dollars. If he buys one glass for 5 dollars, then every second glass costs 5 x 0.6 = 3 dollars. So he will have to pay 3 x 16 = 48 dollars. 80 - 48 is 32. The final answer is 32

Given the following problem, reason and give a final answer to the problem.
Problem: There are 5 apples in the basket. If I take 2 apples, how many apples are left in the basket?
Your response should end with "The final answer is [answer]" where [answer] is the response to the problem.

There are 5 apples in the basket. If I take 2 apples, then I will have 5 - 2 = 3 apples left. The final answer is 3

Given the following problem, reason and give a final answer to the problem.
Problem: There are 5 apples in the basket. If I take 2 apples, how many apples are left in the basket?
Your response should end with "The final answer is [answer]" where [a